<a href="https://colab.research.google.com/github/technologie-intelligente/GAN-MNIST/blob/master/Perso_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Import all data
import numpy as np 
import pandas as pd 
import os
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from PIL import Image
from tqdm import tqdm
import os
import cv2
from keras.applications.densenet import DenseNet121
from keras.models import Sequential
from keras.layers.convolutional import Conv2D 
from keras.layers.convolutional import ZeroPadding2D
from keras import layers
import tensorflow
from keras.optimizers import Adam
from keras.layers import LeakyReLU
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model
from keras.layers import Reshape, UpSampling2D, Activation


Using TensorFlow backend.


In [0]:

class GAN():

    def __init__(self, width, heigth, channels):
        """
        @param width : Width of our images
        @param heigth : Height of our images
        @param channels : Number of channels of our images

        @variable latent_dim : Input dimension

        """
        self.width = width
        self.heigth = heigth
        self.channels = channels
        self.img_shape = (self.width, self.heigth, self.channels)

        optimizer = Adam(0.0002, 0.5)
        
        # Input for our generator
        self.latent_dim = 50

        # Build our discriminator
        self.discriminator = self.discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build our generator
        self.generator = self.generator()


        # The generator takes noise as input and generates imgs
        input_noise = Input(shape=(self.latent_dim,))
        img = self.generator(input_noise)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(input_noise, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)


    def generator (self):
        """ Create a Generator model which takes as input a noise and return
        an image generated.

        Description of our Generator :
        - Dense Layer for the input noise
        - Bunch of convolutional layer to create and generate data

        @return Keras Model Generator
        """
        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        # Scalling up the image (resizing)
        model.add(UpSampling2D())

        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        
        model.add(Conv2D(32, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))

        model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        # Create our input layer with the latent dimension
        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))

        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))

        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))

        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))

        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)


    def train(self, train_images_path, epochs, batch_size=128, save_interval=20):
        
        # Load the dataset
        X_train = train_images #get_images_from_array_path(train_images_path, self.width)
#         X_train = X_train / 127.5 - 1
        
        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Sample noise and generate a batch of new images
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("IT20_image_it_%d.png" % epoch)
        plt.close()

In [0]:
(train_images, train_labels), (_, _) = tf.keras.datasets.mnist.load_data()
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
train_images = (train_images - 127.5) / 127.5 # Normalize the images to [-1, 1]

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
gan = GAN(28, 28, 1)
gan.train(train_images, 10000)

W0810 11:11:48.396706 140488751703936 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0810 11:11:48.399016 140488751703936 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0810 11:11:48.403804 140488751703936 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0810 11:11:48.448212 140488751703936 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0810 11:11:48.459501 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 8, 8, 64)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 8, 64)          256       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 64)          0         
__________

W0810 11:11:49.413058 140488751703936 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0810 11:11:49.426427 140488751703936 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0810 11:11:49.471688 140488751703936 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:2018: The name tf.image.resize_nearest_neighbor is deprecated. Please use tf.compat.v1.image.resize_nearest_neighbor instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 6272)              319872    
_________________________________________________________________
reshape_1 (Reshape)          (None, 7, 7, 128)         0         
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 14, 64)        73792     
_________________________________________________________________
batch_normalization_4 (Batch (None, 14, 14, 64)        256       
_________________________________________________________________
activation_1 (Activation)    (None, 14, 14, 64)        0         
_________________________________________________________________
up_sampling2d_2 (UpSampling2 (None, 28, 28, 64)        0         
__________

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 1.108584, acc.: 33.20%] [G loss: 0.810987]


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D loss: 0.575254, acc.: 70.31%] [G loss: 1.258479]
2 [D loss: 0.327731, acc.: 89.06%] [G loss: 1.953081]
3 [D loss: 0.358007, acc.: 85.94%] [G loss: 2.145086]
4 [D loss: 0.325303, acc.: 87.89%] [G loss: 2.383494]
5 [D loss: 0.369533, acc.: 84.38%] [G loss: 2.855884]
6 [D loss: 0.526468, acc.: 74.61%] [G loss: 3.334607]
7 [D loss: 0.713291, acc.: 60.94%] [G loss: 3.512447]
8 [D loss: 0.720758, acc.: 60.94%] [G loss: 3.341285]
9 [D loss: 0.576277, acc.: 68.75%] [G loss: 2.855685]
10 [D loss: 0.441179, acc.: 79.30%] [G loss: 2.416208]
11 [D loss: 0.284994, acc.: 91.02%] [G loss: 1.891740]
12 [D loss: 0.173215, acc.: 94.53%] [G loss: 1.147614]
13 [D loss: 0.103969, acc.: 97.66%] [G loss: 0.769419]
14 [D loss: 0.083278, acc.: 98.83%] [G loss: 0.358279]
15 [D loss: 0.053978, acc.: 100.00%] [G loss: 0.255619]
16 [D loss: 0.088095, acc.: 99.22%] [G loss: 0.277133]
17 [D loss: 0.093709, acc.: 98.83%] [G loss: 0.308634]
18 [D loss: 0.147703, acc.: 95.70%] [G loss: 0.751687]
19 [D loss: 0.2497

In [0]:
!mkdir photos
!mv IT*.png ./photos/
!zip -r ./photos.zip ./photos/

  adding: photos/ (stored 0%)
  adding: photos/IT20_image_it_2400.png (deflated 6%)
  adding: photos/IT20_image_it_3700.png (deflated 6%)
  adding: photos/IT20_image_it_600.png (deflated 5%)
  adding: photos/IT20_image_it_6440.png (deflated 7%)
  adding: photos/IT20_image_it_1960.png (deflated 6%)
  adding: photos/IT20_image_it_7860.png (deflated 7%)
  adding: photos/IT20_image_it_6740.png (deflated 8%)
  adding: photos/IT20_image_it_4400.png (deflated 7%)
  adding: photos/IT20_image_it_80.png (deflated 6%)
  adding: photos/IT20_image_it_4860.png (deflated 7%)
  adding: photos/IT20_image_it_9260.png (deflated 8%)
  adding: photos/IT20_image_it_6000.png (deflated 7%)
  adding: photos/IT20_image_it_1340.png (deflated 5%)
  adding: photos/IT20_image_it_3940.png (deflated 7%)
  adding: photos/IT20_image_it_4800.png (deflated 7%)
  adding: photos/IT20_image_it_4020.png (deflated 7%)
  adding: photos/IT20_image_it_420.png (deflated 5%)
  adding: photos/IT20_image_it_7840.png (deflated 7%)
  